In [9]:
def scaler(df: pd.DataFrame):

    #Creating a copy of our dataframe to scale
    df_scaled = df.copy()

    #Selecting relevant columns to scale (we dropped 'return' since its our target and we dropped 'date')
    columns_to_scale = df_scaled.drop(columns=['return']).columns

    # Scaling our data based
    for column in columns_to_scale:

        if df_scaled[column][0]!= df_scaled[column][1]:
            scaler_standard = StandardScaler()
            df_scaled[column] = scaler_standard.fit_transform(df_scaled[[column]])

        else:
            scaler_robust = RobustScaler()
            df_scaled[column] = scaler_robust.fit_transform(df_scaled[[column]])

    return df_scaled


In [7]:
# importing relevant librairies
import numpy as np
import pandas as pd
import yfinance as yf


def load_csv(ticker: str):
    """
    Creating a function that would allow us to load a csv containing all relevant features of a given ticker
    ticker should be in capital
    Start and end should be in the be following format: %YYYY-%MM-%DD
    """

    #Loading all relevant csv files

    #Loading stock return
    ticker_df = pd.read_csv(f'../raw_data/stocks_return/adj_return_{ticker}.csv')
    ticker_df.rename(columns={'timestamp':'date'}, inplace=True)
    ticker_df.sort_values('date', ascending=True, inplace=True)
    ticker_df.reset_index(inplace=True, drop=True)

    if '1999-12-31' in ticker_df['date'].values:
        ticker_df = ticker_df[ticker_df['date']>'1999-12-31']
        ticker_df.reset_index(inplace=True, drop=True)

    #Loading stocks eps
    ticker_eps = pd.read_csv(f'../raw_data/eps/data_{ticker}.csv', index_col=0)
    ticker_eps['date'] = ticker_eps['reportedDate'].copy()
    ticker_eps.sort_values('date', inplace=True)
    ticker_eps.reset_index(drop = True, inplace=True)
    ticker_eps['reportedDate']= pd.to_datetime(ticker_eps['reportedDate'])
    ticker_eps['year'] = pd.DatetimeIndex(ticker_eps['reportedDate']).year
    first_year = ticker_eps['year'][0]

    if 2000 in ticker_eps['year'].values:
        ticker_eps = ticker_eps[ticker_eps['year']>=2000]
        ticker_eps.reset_index(inplace=True, drop=True)

    ticker_eps.drop(columns=['year', 'fiscalDateEnding', 'estimatedEPS', 'surprise', 'reportedDate'], inplace=True)

    #Loading all macros and commodities
    gold = pd.read_csv('../raw_data/macro/gold.csv', index_col=0, decimal=',')
    us_dollar = pd.read_csv('../raw_data/macro/usd.csv', index_col=0)
    credit_spread = pd.read_csv('../raw_data/macro/us_yields.csv', index_col=0)
    oil = pd.read_csv('../raw_data/macro/oil.csv', index_col=0)
    orders = pd.read_csv('../raw_data/macro/orders.csv', index_col=0)
    ffunds_rate = pd.read_csv('../raw_data/macro/fed_funds.csv', index_col=0)
    unemployment = pd.read_csv('../raw_data/macro/unemployment.csv', index_col=0)
    inflation_expectation = pd.read_csv('../raw_data/macro/inflation_expectation.csv', index_col=0)
    non_farm_payroll = pd.read_csv('../raw_data/macro/non_farm_payroll.csv', index_col=0)
    cpi = pd.read_csv('../raw_data/macro/cpi.csv', index_col=0)
    retails = pd.read_csv('../raw_data/macro/retail_sales.csv', index_col=0)
    gdp_capita = pd.read_csv('../raw_data/macro/gdp_per_capita.csv', index_col=0)


    #Creating our final df and merging with relevant files
    final_df = ticker_df.copy()
    final_df = final_df.merge(ticker_eps, how='outer', on='date')
    final_df = final_df.merge(credit_spread, how='outer')
    final_df = final_df.merge(oil, how='outer', on='date')
    final_df = final_df.merge(orders, how='outer')
    final_df = final_df.merge(ffunds_rate, how='outer')
    final_df = final_df.merge(unemployment, how='outer')
    final_df = final_df.merge(inflation_expectation, how='outer')
    final_df = final_df.merge(non_farm_payroll, how='outer')
    final_df = final_df.merge(cpi, how='outer')
    final_df = final_df.merge(retails, how='outer')
    final_df = final_df.merge(gdp_capita, how='outer')
    final_df = final_df.merge(gold, how='outer', on='date')
    final_df = final_df.merge(us_dollar, how='outer', on='date')


    #Sorting by chronological order, resetting index and changing the colummn name for the CPI
    final_df = final_df.sort_values('date', ascending=True)
    final_df.reset_index(drop=True, inplace=True)
    final_df.rename(columns={'CPI':'cpi'}, inplace=True)
    final_df['date']= pd.to_datetime(final_df['date'])
    final_df['year'] = pd.DatetimeIndex(final_df['date']).year

    #Since not all features have daily data, we forward filled the missing values
    final_df['orders'] = final_df['orders'].fillna(method='ffill')
    final_df['retail_sales'] = final_df['retail_sales'].fillna(method='ffill')
    final_df['gdp_per_capita'] = final_df['gdp_per_capita'].fillna(method='ffill')
    final_df['cpi'] = final_df['cpi'].fillna(method='ffill')
    final_df['non_farm_payroll'] = final_df['non_farm_payroll'].fillna(method='ffill')
    final_df['inf_exp'] = final_df['inf_exp'].fillna(method='ffill')
    final_df['unemployment_rate'] = final_df['unemployment_rate'].fillna(method='ffill')
    final_df['reportedEPS'] = final_df['reportedEPS'].fillna(method='ffill')
    final_df['surprisePercentage'] = final_df['surprisePercentage'].replace(['None'], '0')
    final_df['surprisePercentage'] = final_df['surprisePercentage'].replace([np.nan], '0')

    #Changing the dtypes of relevant features such as surprisePercentage and gold_price
    final_df["surprisePercentage"] = pd.to_numeric(final_df["surprisePercentage"])

    # Chaging the unemployement rate to decimal
    final_df['unemployment_rate'] = final_df['unemployment_rate']/100
    final_df['surprisePercentage'] = final_df['surprisePercentage']/100

    #splitting our df
    len_one_year_before = final_df[(final_df['year']>=first_year-1) & (final_df['year']<first_year)].shape[0]
    not_needed_index = final_df[(final_df['year'] >= 1999) & (final_df['year'] < first_year-1)].shape[0]

    first_split = final_df.iloc[not_needed_index:not_needed_index+len_one_year_before]

    second_split = final_df.iloc[not_needed_index + len_one_year_before:]
    second_split.dropna(inplace=True)

    final_df2 = pd.concat([first_split, second_split], axis=0)

    final_df2.reset_index(drop=True, inplace=True)

    return final_df2


def features_creation(ticker: str, high_low_ratio: bool = True, volatility: bool = True, momentum: bool = True, distance: bool = True, volume: bool = True, price_eps_ratio: bool = True,
                      momentum_eps_ratio: bool = True, gold_return: bool = True, oil_return: bool = True, usd_return: bool = True, cpi_return: bool = True, period:int = 250,
                      gdp_return: bool = True, ten_year_return: bool = True, two_year_return: bool = True, spread_return: bool = True, volume_momentum: bool = True,
                      non_farm_payroll_return: bool = True, unemployement_return: bool = True, days:list = [5, 10, 20] ):
    """
    Function that adds features to a dataframe
    days should be a list containing the number of days to consider for calculating the volatitliy, momentum, distance and and custom volume
    in order to calculate the momentum/eps ratio, momentum should be True
    """

    final_df = load_csv(ticker=ticker)

    final_df['return'] = final_df['adjusted_close'].pct_change()
    final_df['return'][0]=0

    if high_low_ratio:
        final_df['high/low'] = final_df['high']/final_df['low'] - 1 # max variation in %

    if volatility:
        for day in days:
            final_df[f'volatility_{day}days'] = final_df['return'].rolling(day).std().shift(1)

    if momentum:
        for day in days:
            final_df[f'momentum_{day}days'] = final_df['return'].rolling(day).mean().shift(1)

    if distance:
        for day in days:
            final_df[f'distance_{day}days'] = (final_df['return'] - final_df['return'].rolling(day).mean()).shift(1)

    if volume:
        for day in days:
            final_df[f'volume_{day}days'] = (final_df['volume'].rolling(day).mean()/final_df['volume']).shift(1)

    if volume_momentum:
        for day in days:
            final_df[f'volume_momentum_{day}days'] = final_df['volume'].rolling(day).mean().shift(1)

    if price_eps_ratio:
        final_df['price/eps'] = final_df['adjusted_close']/final_df['reportedEPS']

    if momentum_eps_ratio:
        for day in days:
            final_df[f'momentum_{day}days/eps']=final_df[f'momentum_{day}days']/final_df['reportedEPS']

    if gold_return:
        final_df['gold_return'] = final_df['gold_price'].pct_change()

    if ten_year_return:
        final_df['10Y_return'] = final_df['10Y_yield'].pct_change()

    if two_year_return:
        final_df['2Y_return'] = final_df['2Y_yield'].pct_change()

    if spread_return:
        final_df['spread_return'] = final_df['10_2_spread'].pct_change()
        final_df['spread_return'].replace([np.inf, -np.inf], 0, inplace=True)

    if oil_return:
        final_df['oil_return'] = final_df['oil_price'].pct_change()

    if usd_return:
        final_df['usd_return'] = final_df['usd_price'].pct_change()

    if unemployement_return:
        final_df['unemployement_return'] = final_df['unemployment_rate'].pct_change()

    if cpi_return:
        final_df['cpi_return'] = final_df['cpi'].pct_change(periods=30)

    if non_farm_payroll_return:
        final_df['non_farm_payroll_return'] = final_df['non_farm_payroll'].pct_change(periods=30)

    if gdp_return:
        final_df['gdp_return'] = final_df['gdp_per_capita'].pct_change(periods=period)

    final_df.drop(columns=['high', 'low','open', 'close', 'adjusted_close', 'year'], inplace=True)
    final_df.dropna(inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    return final_df


def clean_data(df: pd.DataFrame):
    final_df = df.copy()

    final_df.dropna(inplace=True)
    final_df.drop(columns='date', inplace=True)
    final_df.reset_index(drop=True, inplace=True)

    return final_df

def ready_to_train_df(ticker:str):

    loaded_features_df = features_creation(ticker=ticker)
    cleaned_df = clean_data(loaded_features_df)
    scaled_df = scaler(cleaned_df)

    return scaled_df


def return_tickers():
    """
    Function to return in list of all S&P500 tickers
    """

    tickers =  ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'BRK.B', 'TSLA', 'UNH', 'XOM', 'JNJ', 'WMT', 'NVDA', 'JPM', 'V', 'CVX', 'PG', 'LLY', 'MA', 'HD', 'META', 'BAC', 'ABBV', 'PFE', 'KO', 'MRK', 'PEP', 'COST', 'ORCL', 'AVGO', 'TMO', 'MCD', 'CSCO', 'ACN', 'DHR', 'TMUS', 'ABT', 'WFC', 'DIS', 'LIN', 'NEE', 'BMY', 'NKE', 'VZ', 'TXN', 'UPS', 'COP', 'ADBE', 'CMCSA', 'CRM', 'PM', 'MS', 'AMGN', 'SCHW', 'HON', 'RTX', 'QCOM', 'T', 'IBM',
 'DE', 'CVS', 'LOW', 'GS', 'UNP', 'NFLX', 'LMT', 'CAT', 'AMD', 'INTC', 'ELV', 'SPGI', 'AXP', 'SBUX', 'INTU', 'BLK', 'ADP', 'GILD', 'PLD', 'MDT', 'BA', 'AMT', 'CI', 'GE', 'TJX', 'ISRG', 'C', 'AMAT', 'PYPL', 'MDLZ', 'CB', 'SYK', 'ADI', 'MMC', 'EOG', 'NOW', 'VRTX', 'MO', 'NOC', 'EL', 'REGN', 'PGR', 'BKNG', 'DUK', 'TGT', 'SLB', 'SO', 'MMM', 'ITW', 'ZTS', 'GD', 'APD', 'HUM', 'MRNA', 'BDX', 'CSX', 'WM', 'PNC', 'HCA', 'ETN', 'USB', 'FISV', 'SHW', 'OXY', 'CL', 'MU', 'CME', 'AON', 'LRCX', 'BSX', 'EQIX', 'TFC', 'PXD', 'CHTR', 'CCI', 'MET', 'ATVI', 'ICE', 'MPC', 'NSC', 'DG', 'GM', 'EMR', 'F', 'KLAC', 'MCO', 'FCX', 'KDP', 'MNST', 'MCK', 'VLO', 'ORLY', 'ADM', 'PSX',
 'PSA', 'SRE', 'SNPS', 'MAR', 'D', 'GIS', 'AEP', 'AZO', 'KHC', 'APH', 'HSY', 'CNC', 'CTVA', 'EW', 'CTAS', 'A', 'ROP', 'JCI', 'CDNS', 'FDX', 'NXPI', 'AIG', 'KMB', 'AFL', 'HES', 'MSI', 'PAYX', 'DVN', 'TRV', 'BIIB', 'DXCM', 'SYY', 'LHX', 'RSG', 'ENPH', 'ECL', 'ADSK', 'MCHP', 'ANET', 'KMI', 'CMG', 'FTNT', 'AJG', 'STZ', 'TT', 'WMB', 'MSCI', 'O', 'IQV', 'TEL', 'ROST', 'PRU', 'EXC', 'PH', 'FIS', 'SPG', 'COF', 'NUE', 'XEL', 'HLT', 'CARR', 'PCAR', 'BK', 'NEM', 'DOW', 'EA', 'WBA', 'DD', 'ALL', 'YUM', 'AMP', 'CMI', 'ILMN', 'BF.B', 'TDG', 'IDXX', 'ED', 'KR', 'ABC', 'DLTR', 'RMD', 'ALB', 'HAL', 'NDAQ', 'LVS', 'ODFL', 'WELL', 'AME', 'CSGP', 'OTIS', 'MTD', 'SBAC', 'ON', 'VICI', 'DLR', 'CEG', 'KEYS', 'PPG', 'WEC', 'CTSH', 'ROK', 'GWW', 'PCG', 'HPQ', 'FAST', 'DFS', 'MTB', 'PEG', 'OKE', 'DHI', 'APTV', 'BKR', 'GLW', 'LYB', 'ES', 'BAX', 'STT', 'VRSK', 'TROW', 'WBD', 'AWK', 'IT', 'GPN', 'HRL', 'FANG', 'WTW', 'RJF', 'GPC', 'IFF', 'CDW', 'TSCO', 'FITB', 'ARE', 'URI', 'ZBH', 'K', 'LEN', 'EBAY', 'EIX', 'CBRE', 'EFX', 'VMC', 'TSN',
 'HIG', 'FTV', 'WY', 'EQR', 'AVB', 'MKC', 'ETR', 'LUV', 'ULTA', 'AEE', 'MLM', 'FE', 'PFG', 'FRC', 'DTE', 'DAL', 'HBAN', 'IR', 'CTRA', 'ANSS', 'ACGL', 'PPL', 'RF', 'VRSN', 'LH', 'EXR', 'PWR', 'CF', 'CAH', 'CFG', 'XYL', 'HPE', 'EPAM', 'DOV', 'WAT', 'WRB', 'TDY', 'PAYC', 'ROL', 'NTRS', 'MRO', 'CNP', 'INVH', 'CHD', 'AES', 'MOH', 'JBHT', 'MAA', 'BBY', 'CLX', 'HOLX', 'WAB', 'DRI', 'EXPD', 'STE', 'AMCR', 'VTR', 'IEX', 'CAG', 'CMS', 'KEY', 'MPWR', 'BALL', 'J', 'BR', 'GRMN', 'PKI', 'TTWO', 'INCY', 'FDS', 'MOS', 'SEDG', 'CINF', 'ABMD', 'DGX', 'WST', 'ATO', 'TRGP', 'BRO', 'SYF', 'FOX', 'FOXA', 'NTAP', 'FMC', 'EQT', 'OMC', 'SJM', 'LYV', 'CPB', 'HWM', 'CPRT', 'AVY', 'IRM', 'COO', 'ALGN', 'SWKS', 'EXPE', 'RCL', 'ETSY', 'APA', 'GEN', 'TXT', 'LDOS', 'LKQ', 'TER', 'PTC', 'TRMB', 'AKAM', 'NVR', 'UAL', 'LNT', 'FLT', 'KIM', 'ZBRA', 'TYL', 'DPZ', 'JKHY', 'MGM', 'ESS', 'L', 'PEAK', 'MTCH', 'NDSN', 'EVRG', 'VTRS', 'IPG', 'BEN', 'CBOE', 'TECH', 'SIVB', 'VFC', 'IP', 'HST', 'UDR', 'POOL', 'RE', 'PARA', 'SNA', 'CPT', 'LW', 'PKG', 'CRL', 'SWK', 'BIO', 'WDC', 'CHRW', 'STX', 'MAS', 'GL', 'CE', 'REG', 'NI', 'BXP', 'HSIC', 'CCL', 'TFX', 'CZR', 'NWS', 'NWSA', 'KMX', 'EMN', 'JNPR', 'PHM', 'CDAY', 'ALLE', 'QRVO', 'BWA', 'NRG', 'MKTX', 'WRK', 'UHS', 'FFIV', 'AOS', 'CMA', 'AAL', 'BBWI', 'HII', 'AAP', 'TPR', 'FRT', 'IVZ', 'PNW',
 'HAS', 'WYNN', 'FBHS', 'SBNY', 'DISH', 'RHI', 'WHR', 'ZION', 'CTLT', 'PNR', 'SEE', 'RL', 'NCLH', 'DXC', 'GNRC', 'AIZ', 'XRAY', 'LNC', 'DVA', 'MHK', 'OGN', 'LUMN', 'ALK', 'NWL', 'VNO', 'TAP']

    return tickers


In [10]:
ready_to_train_df('AAPL')

/tmp/ipykernel_9235/2252850877.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  second_split.dropna(inplace=True)
/tmp/ipykernel_9235/2252850877.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['return'][0]=0


,volume,dividend_amount,split_coefficient,reportedEPS,surprisePercentage,10Y_yield,2Y_yield,10_2_spread,oil_price,orders,fed_funds,unemployment_rate,inf_exp,non_farm_payroll,cpi,retail_sales,gdp_per_capita,gold_price,usd_price,return,high/low,volatility_5days,volatility_10days,volatility_20days,momentum_5days,momentum_10days,momentum_20days,distance_5days,distance_10days,distance_20days,volume_5days,volume_10days,volume_20days,volume_momentum_5days,volume_momentum_10days,volume_momentum_20days,price/eps,momentum_5days/eps,momentum_10days/eps,momentum_20days/eps,gold_return,10Y_return,2Y_return,spread_return,oil_return,usd_return,unemployement_return,cpi_return,non_farm_payroll_return,gdp_return
0,-0.346003,0.0,0.0,-0.520563,0.0,1.683449,2.008561,-1.025447,-1.302865,-1.106639,2.815027,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620106,1.792221,0.111838,1.983617,2.728459,2.969448,2.045750,-0.491008,1.738754,1.683579,-0.725595,-1.461834,-1.257414,-1.224829,-1.146544,-1.412197,-0.772641,-0.792025,-0.876532,-0.986475,0.062426,-0.619727,-0.682886,0.433807,-0.548798,-0.525452,0.685932,0.415832,-1.588763,0.0,0.661811,-2.436691,-0.129723
1,-0.614623,0.0,0.0,-0.520563,0.0,1.741110,2.029087,-0.979457,-1.237085,-1.106639,2.808797,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620504,1.858971,0.043339,1.038557,4.023115,3.177779,2.666917,2.544799,1.907877,2.682564,4.210961,4.500903,4.393001,-1.610149,-1.688337,-1.797802,-0.690418,-0.765302,-0.846398,-1.000526,-0.844069,-0.669428,-0.995584,-0.106335,0.535720,0.104186,0.479318,1.062331,1.322936,0.0,0.661811,-2.436691,-0.129723
2,-0.768118,0.0,0.0,-0.520563,0.0,1.790534,2.029087,-0.910473,-1.233602,-1.106639,2.858633,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.616334,1.820033,-0.012821,1.976073,3.436255,3.184096,2.643568,1.653038,1.925050,3.301135,1.267393,1.323638,1.159753,-0.501205,-0.811839,-1.063774,-0.693862,-0.762872,-0.832934,-0.996189,-0.577788,-0.674475,-1.189207,0.692156,0.451417,-0.024290,0.654627,0.054477,-0.768696,0.0,0.661811,-2.436691,-0.129723
3,-0.553266,0.0,0.0,-0.520563,0.0,1.831721,2.076982,-0.933468,-1.254884,-1.106639,2.839945,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.617128,1.820033,0.064935,3.838197,2.877404,2.955185,2.688142,2.310944,2.324240,2.895104,-1.814998,-1.404638,-1.287929,1.138698,0.419339,-0.142960,-0.681654,-0.755899,-0.834472,-1.017874,-0.774240,-0.791786,-1.062113,-0.174732,0.370081,0.273594,-0.195624,-0.322929,-0.001263,0.0,0.661811,-2.436691,-0.129723
4,-0.650157,0.0,0.0,-0.520563,0.0,1.856433,2.070140,-0.887478,-1.262623,-1.106639,2.889780,-0.521739,0.000000,-0.508049,-1.199577,-1.132819,-1.112398,-1.620305,1.953532,0.000000,1.641733,2.864462,3.142905,2.723276,3.721787,3.094854,3.780943,1.334317,1.948782,2.029040,-0.352958,-0.886348,-1.213817,-0.608695,-0.722924,-0.817330,-1.017874,-1.195519,-1.018250,-1.339395,-0.588911,0.215807,-0.066227,0.398967,-0.118665,2.639928,0.0,0.661811,-2.436691,-0.129723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5429,1.067172,0.0,0.0,1.920188,0.0,0.489039,1.755402,-2.267170,0.833050,1.298006,1.506848,-0.739130,3.333333,1.418862,1.659773,2.233977,1.796868,1.376334,1.452910,-0.008331,-0.521921,2.045629,1.623760,1.321049,1.518759,-0.406861,0.360028,-0.217573,0.613723,0.391685,-0.254195,-0.133902,-0.204252,2.035810,2.266810,2.290988,0.612943,-0.036839,-0.048508,-0.062591,-0.268393,-1.376304,-0.116800,0.927587,-0.271928,-0.229311,0.0,-0.426518,-0.373957,-0.999994
5430,1.607330,0.0,0.0,1.920188,0.0,0.571412,1.810139,-2.244175,0.680982,1.298006,1.506848,-0.739130,3.333333,1.418862,1.659773,2.233977,1.796868,1.350718,1.490921,0.012971,0.496786,2.142008,1.593324,1.327480,1.263072,-0.273751,0.179782,-1.089147,-0.364285,-0.481144,0.801119,1.008195,0.818930,1.852762,2.207812,2.225604,0.629484,-0.037413,-0.04